In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Data/dataFlight_version_3.csv", index_col=0)
df.head(3).append(df.tail(3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,DayofMonth,DayOfWeek,Reporting_Airline,Origin,Dest,ArrDelay
Month,,,,,,
5,28,4,AirTran Airways Corporation,General Mitchell International Airport,Orlando International Airport,0.0
6,29,6,Envoy Air - MQ,Grand Junction Regional Airport,Dallas/Fort Worth International Airport,-3.0
8,31,2,Delta Airlines,Los Angeles International Airport,Detroit Metropolitan Wayne County Airport,-20.0
11,14,5,US Airways Inc.,Charleston International Airport,Charlotte Douglas International Airport,-11.0
5,15,2,Southwest Airlines,Chicago Midway International Airport,Nashville International Airport,-3.0
4,29,2,United Airlines - UA,Washington Dulles International Airport,Chicago O'Hare International Airport,-14.0


# Clustering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Créez une copie du DataFrame pour conserver les données d'origine intactes
df_copy = df.copy()

# Attribuer la catégorie 0 aux vols à l'heure (ArrDelay négatif ou nul)
df_copy['ArrDelayCategory'] = np.where(df_copy['ArrDelay'] <= 0, 0, np.nan)

# Filtrer les vols ayant un retard positif
positive_delay = df_copy[df_copy['ArrDelay'] > 0]

# Préparer les données pour le clustering
X = positive_delay['ArrDelay'].values.reshape(-1, 1)

# Test de différentes valeurs de k pour trouver le nombre optimal de clusters
k_values = list(range(2, 8))  # Test de 2 à 6 clusters
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Pour k={k}, le score de silhouette moyen est : {silhouette_avg}")

# Trouver le nombre optimal de clusters en fonction du score de silhouette
optimal_k = k_values[np.argmax(silhouette_scores)]
print(f"\nLe nombre optimal de clusters est : {optimal_k}")

# Création du modèle K-means avec le nombre optimal de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=0)
positive_delay['ArrDelayCategory'] = kmeans.fit_predict(X) + 1  # Ajouter 1 pour que les catégories commencent à 1

# Fusionner les résultats du clustering avec le DataFrame original
df_copy.update(positive_delay)

# Affichage des catégories et de leurs centres
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"Catégorie {i + 1}: {center[0]} minutes de retard")

# Afficher les catégories et les retards correspondants
print(df_copy[['ArrDelay', 'ArrDelayCategory']])

# Regression linéaire

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

# Supposons que vos données soient dans un DataFrame nommé df
# Définissez les colonnes catégorielles et numériques
categorical_features = ['Reporting_Airline', 'Origin', 'Dest']
numeric_features = ['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime']

# Mélangez les données avant de les découper
df = shuffle(df, random_state=42)

# Divisez les données en ensembles d'apprentissage et de test
X = df.drop('ArrDelay', axis=1)
y = df['ArrDelay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créez un pipeline pour le prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Instancier le modèle LinearRegression
lr_model = LinearRegression()

# Entraîner le modèle
lr_model.fit(X_train, y_train)

# Prédire les retards sur l'ensemble de test
y_pred = lr_model.predict(X_test)

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error: ", mse)
print("R2 score: ", r2)

# Calculer l'exactitude avec une marge d'erreur de 5 minutes
accuracy = np.sum(np.abs(y_test - y_pred) <= 5) / len(y_test)
print("Accuracy with 5 minutes margin: ", accuracy)

# RN

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Supposons que vos données soient dans un DataFrame nommé df
# Définissez les colonnes catégorielles et numériques
categorical_features = ['Reporting_Airline', 'Origin', 'Dest']
numeric_features = ['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime']

# Mélangez les données avant de les découper
df = shuffle(df, random_state=42)

# Divisez les données en ensembles d'apprentissage et de test
X = df.drop('ArrDelay', axis=1)
y = df['ArrDelay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créez un pipeline pour le prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_features)
])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Créez le modèle de réseau de neurones
nn_model = Sequential()
nn_model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(1))

# Compilez le modèle
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
nn_model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Prédire les retards sur l'ensemble de test
y_pred = nn_model.predict(X_test).flatten()

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error: ", mse)
print("R2 score: ", r2)

# Calculer l'exactitude avec une marge d'erreur de 5 minutes
accuracy = np.sum(np.abs(y_test - y_pred) <= 5) / len(y_test)
print("Accuracy with 5 minutes margin: ", accuracy)

# SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

# Supposons que vos données soient dans un DataFrame nommé df
# Définissez les colonnes catégorielles et numériques
categorical_features = ['Reporting_Airline', 'Origin', 'Dest']
numeric_features = ['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime']

# Remplacez la colonne Reporting_Airline par son code numérique
#airline_mapping = {'AirTran Airways Corporation': 'FL'}
#df['Reporting_Airline'] = df['Reporting_Airline'].replace(airline_mapping)

# Mélangez les données avant de les découper
df = shuffle(df, random_state=42)

# Divisez les données en ensembles d'apprentissage et de test
X = df.drop('ArrDelay', axis=1)
y = df['ArrDelay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créez un pipeline pour le prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Instancier le modèle SVM avec un noyau linéaire
svm_model = SVR(kernel='linear')

# Entraîner le modèle SVM
svm_model.fit(X_train, y_train)

# Prédire les retards sur l'ensemble de test
y_pred = svm_model.predict(X_test)

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error: ", mse)
print("R2 score: ", r2)

# RandomForest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

# Supposons que vos données soient dans un DataFrame nommé df
# Définissez les colonnes catégorielles et numériques
categorical_features = ['Reporting_Airline', 'Origin', 'Dest']
numeric_features = ['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime']

# Mélangez les données avant de les découper
df = shuffle(df, random_state=42)

# Divisez les données en ensembles d'apprentissage et de test
X = df.drop('ArrDelay', axis=1)
y = df['ArrDelay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créez un pipeline pour le prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Instancier le modèle RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Entraîner le modèle
rf_model.fit(X_train, y_train)

# Prédire les retards sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean squared error: ", mse)
print("R2 score: ", r2)

# Calculer l'exactitude avec une marge d'erreur de 5 minutes
accuracy = np.sum(np.abs(y_test - y_pred) <= 5) / len(y_test)
print("Accuracy with 5 minutes margin: ", accuracy)